In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import seaborn as sns

# Load and clean dataset
file_path = "Book1_vashnvi.xlsx"
data = pd.read_excel(file_path, header=1)  # Load with the second row as the header
data.columns = ['Index', 'ChildsName', 'LORRoom1', 'GrossMotor', 'FineMotor', 'SleepDisorder']  # Rename columns

# Drop unnecessary column
data.drop(columns=['Index'], inplace=True)

# Replace empty strings with NaN and define numeric columns
data.replace("", np.nan, inplace=True)
numeric_cols = ['LORRoom1', 'GrossMotor', 'FineMotor', 'SleepDisorder']

# Convert specified columns to numeric, coercing errors to NaN
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Enhanced imputation with median
imputer = SimpleImputer(strategy='median')
data[numeric_cols] = imputer.fit_transform(data[numeric_cols])

# Display dataset summary
print("\nCleaned Data Overview:")
print(data.info())
print(data.describe())

# Define features and target
X = data[['LORRoom1', 'GrossMotor', 'FineMotor']]
y = data['SleepDisorder']

# Step 1: Polynomial Regression with Stepwise Selection
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_poly = sm.add_constant(X_poly)
poly_model = sm.OLS(y, X_poly).fit()

# Stepwise feature elimination based on p-values
p_threshold = 0.05  # Set the threshold to 0.05
while True:
    p_values = poly_model.pvalues
    max_p_value = p_values.max()  # Find the maximum p-value
    if max_p_value > p_threshold:
        max_p_feature = p_values.idxmax()  # Feature with the highest p-value
        feature_index = p_values.index.get_loc(max_p_feature)
        X_poly = np.delete(X_poly, feature_index, axis=1)  # Remove feature with high p-value
        poly_model = sm.OLS(y, X_poly).fit()  # Re-fit the model with the remaining features
    else:
        break

print("\nPolynomial Regression OLS Summary (Post Feature Selection):")
print(poly_model.summary())

# Step 2: Regularization Techniques (Ridge and Lasso)
ridge = Ridge()
lasso = Lasso()
ridge_params = {'alpha': [1e-6, 1e-3, 1, 10, 100]}
lasso_params = {'alpha': [1e-6, 1e-3, 1, 10, 100]}

ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring='neg_mean_squared_error')
lasso_grid = GridSearchCV(lasso, lasso_params, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(X_poly, y)
lasso_grid.fit(X_poly, y)

print("\nBest Ridge Alpha:", ridge_grid.best_params_)
print("Best Lasso Alpha:", lasso_grid.best_params_)

# Step 3: Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree Regressor
tree_model = DecisionTreeRegressor(max_depth=5, random_state=42)  # You can adjust max_depth to control overfitting
tree_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_tree = tree_model.predict(X_test)

# Calculate the Mean Squared Error (MSE) to evaluate the model's performance
mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f"Decision Tree MSE: {mse_tree:.4f}")

# Plotting the predictions vs true values for better visualization
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_tree, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')  # Identity line
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Decision Tree Regression: True Values vs Predicted Values')
plt.show()

# You can also visualize the decision tree itself
from sklearn.tree import plot_tree

plt.figure(figsize=(12, 8))
plot_tree(tree_model, filled=True, feature_names=X.columns, fontsize=10)
plt.title('Decision Tree Model Visualization')
plt.show()

# Step 4: Improved CNN Approach
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_cnn = X_train.reshape(-1, 1, X_train.shape[1])
X_test_cnn = X_test.reshape(-1, 1, X_test.shape[1])

# Define the CNN model
cnn_model = Sequential([
    Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(1, X_train.shape[1])),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Output layer for regression
])

cnn_model.compile(optimizer=Adam(learning_rate=0.0001), loss=MeanSquaredError())
history = cnn_model.fit(X_train_cnn, y_train, epochs=100, batch_size=8, validation_data=(X_test_cnn, y_test), verbose=1)

# Step 5: Evaluate CNN Model
y_pred_cnn_continuous = cnn_model.predict(X_test_cnn).flatten()
threshold = np.median(y_train)
y_pred_cnn_binary = (y_pred_cnn_continuous > threshold).astype(int)
y_test_binary = (y_test > threshold).astype(int)

accuracy = accuracy_score(y_test_binary, y_pred_cnn_binary)
precision = precision_score(y_test_binary, y_pred_cnn_binary)
recall = recall_score(y_test_binary, y_pred_cnn_binary)
f1 = f1_score(y_test_binary, y_pred_cnn_binary)

print("\nCNN Model Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.show()

# Visualization: Scatter Plots
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.scatter(data['LORRoom1'], data['GrossMotor'], color='blue', alpha=0.5, label='Gross Motor')
plt.xlabel("LORRoom1")
plt.ylabel("Gross Motor Score")
plt.title("LORRoom1 vs Gross Motor")
plt.legend()

plt.subplot(1, 3, 2)
plt.scatter(data['LORRoom1'], data['FineMotor'], color='green', alpha=0.5, label='Fine Motor')
plt.xlabel("LORRoom1")
plt.ylabel("Fine Motor Score")
plt.title("LORRoom1 vs Fine Motor")
plt.legend()

plt.subplot(1, 3, 3)
plt.scatter(data['LORRoom1'], data['SleepDisorder'], color='red', alpha=0.5, label='Sleep Disorder')

plt.xlabel("LORRoom1")
plt.ylabel("Sleep Disorder Score")
plt.title("LORRoom1 vs Sleep Disorder")
plt.legend()

plt.tight_layout()
plt.show()

# Correlation Heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = data[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Heatmap of Numeric Features")
plt.show()